# Data Wrangling

This JSON data is in a strange format.  The columns appear in an array at the top, and then records appear in an array in a separate field.  We need to create a dataframe with this data.

In [24]:
import os
import pandas as pd
import json

f = open("data/2020-7.json")
data = json.load(f)
f.close()

df = pd.DataFrame(columns=data["columns"])


In [28]:

for fname in os.listdir("data"):
    f = open("data/" + fname)
    data = json.load(f)
    f.close()
    df = df.append(data["records"], ignore_index=True)

df.shape

(12231, 20)

In [29]:
df.to_csv("fulldata/data.csv")

In [39]:
df.subdivisiondescr.fillna("NONE",inplace=True)

In [43]:
subdivisions = df.subdivisiondescr.unique()

In [44]:
subdivisions.sort()

In [46]:
subdivisions

array(['435 EAST DRAKE CONDO AMD', 'ABERDEEN AT WATERGLEN',
       'ABERDEEN AT WATERGLEN CONDO SUPP. NO. 1',
       'ABERDEEN AT WATERGLEN CONDO, SUPP NO.9,',
       'ABERDEEN AT WATERGLEN SUPP 11 BLDG J CONDOS FTC (20050010812)',
       'ABERDEEN AT WATERGLEN, SUPP 6 (BLDG G), FTC (20040010844)',
       'ABERDEEN AT WATERGLEN, SUPP NO. 7, FTC (20040035245)',
       'ADER ESTATES PUD', 'ADRIEL HILLS', 'ADRIEL HILLS OVERALL',
       'ADRIEL HILLS TWO', 'ADRIEL VILLAGE', 'ALTA VISTA',
       'ALVIN L MILLER', 'ANDERSON PLACE',
       'APPALOOSA AT LORY ANN ESTATES CONDOS', 'APPLERIDGE PUD',
       'APPLERIDGE TOWNHOMES', 'APPLEWOOD ESTATES',
       'ARAPAHOE FARM TOWNHOMES PUD', 'ARAPAHOE HILLS',
       'ARAPAHOE PINES CONDOS', 'ARROWHEAD II CONDOS BLDG 3 AMNDED MAP',
       'ARTALICE', 'ASHFORD COMMONS PUD', 'ASPEN GROVE CONDOS',
       'ASPEN VILLAGE CONDOS',
       'AUBURN ESTATES CONSERVATION DEVELOPMENT 03-S2143 (20070021442)',
       'AURORA HILLS', 'AUTUMN CREEK CONS DEV, 2ND FLG